In [28]:
import cv2
import numpy as np
import math
import time
import torch
from ultralytics import YOLO


In [29]:
!pip install ultralytics

In [30]:
model = YOLO("best.pt")  # your trained model
vehicle_classes = ['car','truck','bus','bike']

In [31]:
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    return cv2.bitwise_and(img, mask)

def draw_lane_lines(img, left_line, right_line, color=[0, 255, 0]):
    line_img = np.zeros_like(img)
    lane_mask = np.zeros(img.shape[:2], dtype=np.uint8)

    poly_pts = np.array([[
        (left_line[0], left_line[1]),
        (left_line[2], left_line[3]),
        (right_line[2], right_line[3]),
        (right_line[0], right_line[1])
    ]], dtype=np.int32)

    cv2.fillPoly(line_img, poly_pts, color)
    cv2.fillPoly(lane_mask, poly_pts, 255)

    combined = cv2.addWeighted(img, 0.8, line_img, 0.5, 0.0)
    return combined, lane_mask


def lane_pipeline(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)

    height, width = edges.shape
    roi = [(0, height), (width//2, height//2), (width, height)]
    cropped = region_of_interest(edges, np.array([roi], np.int32))

    lines = cv2.HoughLinesP(
        cropped,
        rho=6,
        theta=np.pi / 60,
        threshold=160,
        minLineLength=40,
        maxLineGap=25
    )

    lane_mask = None

    if lines is None:
        return frame, lane_mask

    left_x, left_y = [], []
    right_x, right_y = [], []

    for line in lines:
        x1, y1, x2, y2 = line[0]
        if x2 == x1:
            continue

        slope = (y2 - y1) / (x2 - x1)
        if abs(slope) < 0.5:
            continue

        if slope < 0:
            left_x.extend([x1, x2])
            left_y.extend([y1, y2])
        else:
            right_x.extend([x1, x2])
            right_y.extend([y1, y2])

    if not left_x or not right_x:
        return frame, lane_mask

    min_y = int(height * 0.6)
    max_y = height

    left_fit = np.poly1d(np.polyfit(left_y, left_x, 1))
    right_fit = np.poly1d(np.polyfit(right_y, right_x, 1))

    left_line = [
        int(left_fit(max_y)), max_y,
        int(left_fit(min_y)), min_y
    ]

    right_line = [
        int(right_fit(max_y)), max_y,
        int(right_fit(min_y)), min_y
    ]

    frame, lane_mask = draw_lane_lines(frame, left_line, right_line)

    return frame, lane_mask





In [32]:
def estimate_distance(bbox_width):
    focal_length = 1000
    known_width = 2.0  # average vehicle width (meters)
    return (known_width * focal_length) / bbox_width


In [33]:
locked_box = None
lock_frames = 0
MAX_LOCK_FRAMES = 5


In [34]:
cap = cv2.VideoCapture("express.mp4")  # or 0 for webcam

In [35]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    resized_frame = cv2.resize(frame, (1280, 720))

    # 1️⃣ Lane detection + mask
    # 1️⃣ Lane detection + mask
    lane_frame, lane_mask = lane_pipeline(resized_frame)

    # 2️⃣ YOLO detection
    results = model(resized_frame, verbose=False)
    vehicle_detected = False

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            class_name = model.names[cls]

            if class_name in vehicle_classes and conf >= 0.5:

                # 🔹 Check if vehicle is inside lane
                if lane_mask is not None:
                    cx = int((x1 + x2) / 2)
                    cy = int((y1 + y2) / 2)
                    h, w = lane_mask.shape

                    if cx < 0 or cy < 0 or cx >= w or cy >= h:
                        continue
                    if lane_mask[cy, cx] == 0:
                        continue

                # 🔹 Lock this vehicle
                vehicle_detected = True
                locked_box = (x1, y1, x2, y2, class_name)
                lock_frames = 0
                break

    # 3️⃣ Target lock fallback
    if not vehicle_detected and locked_box is not None:
        lock_frames += 1
        if lock_frames > MAX_LOCK_FRAMES:
            locked_box = None

    # 4️⃣ Draw locked vehicle + distance + warning
    if locked_box:
        x1, y1, x2, y2, class_name = locked_box
        bbox_width = x2 - x1
        bbox_height = y2 - y1

        distance = estimate_distance(bbox_width)

        cv2.rectangle(lane_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(
            lane_frame,
            f"{class_name} {distance:.2f} m",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

        # Warning logic
        if distance < 1.0:
            msg = "STOP"
        elif distance < 2.5:
            msg = "SLOW"
        else:
            msg = "SAFE"

        cv2.putText(
            lane_frame,
            msg,
            (50, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            3
        )

    cv2.imshow("Lane + Vehicles + Distance", lane_frame)

    cv2.waitKey(1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [36]:
print(cap)


< cv2.VideoCapture 000001FF2CB672F0>


In [ ]:
if not cap.isOpened():
    print("Error: Video not opened")


In [10]:
import cv2
from ultralytics import YOLO

# Load model
model = YOLO("yolov8n.pt")

# Camera
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 30)

# Safety
if not cap.isOpened():
    print("❌ Camera not accessible")
    exit()

locked_box = None
lock_frames = 0
MAX_LOCK_FRAMES = 15

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1280, 720))

    lane_frame, lane_mask = lane_pipeline(frame)

    results = model(frame, conf=0.5, verbose=False)
    vehicle_detected = False

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            class_name = model.names[cls]

            if class_name not in vehicle_classes:
                continue

            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            if lane_mask is not None:
                h, w = lane_mask.shape
                if not (0 <= cx < w and 0 <= cy < h):
                    continue
                if lane_mask[cy, cx] == 0:
                    continue

            locked_box = (x1, y1, x2, y2, class_name)
            lock_frames = 0
            vehicle_detected = True
            break

        if vehicle_detected:
            break

    if not vehicle_detected and locked_box is not None:
        lock_frames += 1
        if lock_frames > MAX_LOCK_FRAMES:
            locked_box = None

    if locked_box is not None:
        x1, y1, x2, y2, class_name = locked_box
        distance = estimate_distance(x2 - x1)

        cv2.rectangle(lane_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(lane_frame,
                    f"{class_name} {distance:.2f} m",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        msg = "STOP" if distance < 1 else "SLOW" if distance < 2.5 else "SAFE"
        cv2.putText(lane_frame, msg, (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    cv2.imshow("Real-Time Camera Detection", lane_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
